In [ ]:
AHP map matching for KCMMN

In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from datetime import datetime
from IMM import IMM
from Junction_Condition import JC
from MMJ import MMJ

# import sys
# sys.path.append('..')

In [3]:
def AHPMM(trajectory_data, nodes_data, edges_data):
    matched_links = []
    matched_links_no_duplicate = []
    iter, previous_edge = IMM(trajectory_data, edges_data, iter=0)
    previous_edge['method'] = ['IMM']
    matched_links.append(previous_edge)
    matched_links_no_duplicate.append(previous_edge)
    while iter < len(trajectory_data):
        current_position = trajectory_data.iloc[[iter]]
        previous_position = trajectory_data.iloc[[iter-1]]
        if JC(nodes_data, edges_data, current_position, previous_position, previous_edge):
            previous_edge['method'] = ['JC']
            matched_links.append(previous_edge)
            iter = iter + 1
        else:
            temp = previous_edge
            iter, previous_edge = MMJ(trajectory_data, nodes_data, edges_data, current_position, previous_edge, iter)
            previous_edge['method'] = ['SMM']
            matched_links.append(previous_edge)
            if (previous_edge.index[0] != temp.index[0]) and (previous_edge.iloc[0].method == 'SMM'):
                matched_links_no_duplicate.append(previous_edge)

    return matched_links, matched_links_no_duplicate

In [4]:
import sys
sys.path.append('..')
import preprocessing.downloader as dl


In [6]:
%%time
dataset_dir = '../Data/map-matching-dataset/'
speed_direction_approixmation_dir = '../kcmmn/approximation'

matched_links_results = []
matched_links_no_duplicate_results = []

testing_range = range(93, 94)
for i in tqdm(testing_range):
    print(i)
    trajectory_number = str(i)
    trajectory_data, nodes_data, edges_data = dl.KCMMN_input_for_fuzzy_AHP(trajectory_number, dataset_dir=dataset_dir,speed_direction_approximation_dir=speed_direction_approixmation_dir)
    for j in range(len(trajectory_data)):
        if (trajectory_data['GPS Bearing'][j] <= (np.pi / 2)) and (trajectory_data['GPS Bearing'][j] >= 0 ):
            trajectory_data['GPS Bearing'][j] = (np.pi / 2 - trajectory_data['GPS Bearing'][j]) * 360 / (2 * np.pi)
        else:
            trajectory_data['GPS Bearing'][j] = (5 * np.pi / 2 - trajectory_data['GPS Bearing'][j]) * 360 / (2 * np.pi)    
    matched_links, matched_links_no_duplicate = AHPMM(trajectory_data, nodes_data, edges_data)

    matched_links_results.append(matched_links)
    matched_links_no_duplicate_results.append(matched_links_no_duplicate)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

80


<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  5%|████                                                                              | 1/20 [03:06<59:05, 186.58s/it]

81


<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 10%|████████                                                                        | 2/20 [07:15<1:07:03, 223.50s/it]

...

98


<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 95%|███████████████████████████████████████████████████████████████████████████    | 19/20 [1:19:27<04:35, 275.23s/it]

99


<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|███████████████████████████████████████████████████████████████████████████████| 20/20 [1:25:50<00:00, 257.50s/it]

CPU times: total: 1h 20min 42s
Wall time: 1h 25min 50s


In [10]:
import os
import dask.bag as db
import algorithms.metric_mm as metric_mm
import preprocessing.constants as constants

ground_truth_dir = dataset_dir

errors2 = []
gts = []
for i in tqdm(range(len(testing_range))):
    trajectory_number = str(i+80)
    gt = dl.load_kcmmn_ground_truth(trajectory_number, dataset_dir=ground_truth_dir)
    matched_edge_df = pd.concat(matched_links_no_duplicate_results[i])
    error = mm_utils.evaluate(matched_edge_df,
        gt,
        matchid = "geometry")
    errors2.append(error)
    gts.append(gt)

print('Error about uniqufied matched_edge_df: ', np.mean(errors2))

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:09<00:00,  2.01it/s]

Error about uniqufied matched_edge_df:  0.19280874896637643


In [40]:
fig = plt.figure(1)
plt.boxplot(errors2,vert=False)
plt.title('AHP (uniqufied) Error %s on ' 
          + str(20) 
          + ' test cases \n(Average error = ' 
          + str(np.average(errors2)) + ')')
plt.show()


In [72]:
# visualization
%matplotlib tk

idx = 0

# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = trajectory_data['geometry'].to_frame()



# this plot all the road system 
edges_data.plot(ax=ax, color = "#bfe4ff")

# ground truth
gts[19].plot(ax = ax, linewidth=4, color = 'Yellow')

# result
pd.concat(matched_links_results[19]).plot(ax = ax, color = "Black")
point_locs.plot(ax = ax, markersize = 5, color="Red")
plt.title("Some jumps", y=-0.25)

# matched_points_df.plot(ax = ax, color = "Red")

Text(0.5, -0.25, 'Some jumps')